# Regression LSTM with best parameters
    find the best prediction window to apply w/ lr = 0.0001

# Import libraries

In [7]:
!pip install pandas
!pip install sklearn
!pip install matplotlib

import pandas as pd;
import numpy as np;
import sklearn
import matplotlib.pyplot as plt
import time

date =  [2018010000, 
         2018030000, 
         2018050000,
         2018070000, 
         2018090000, 
         2018110000]

# parameters
steps = 96
n_hidden = 2
units = 150
batch_size = 336
epochs = 100
features_num = 14

# lists to append results
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []
y_pred_list = []
time_count = []

# Import keras libraries, packages and data:

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers

# import data
data_full = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# Create loop for different dates:

In [ ]:
# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# design the LSTM
def regressor_tunning(kernel_initializer = 'he_uniform',
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    if n_hidden == 1:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(LSTM(units = units, 
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    optimizer = optimizers.RMSprop()
    model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
    return model

  
# LOOP STARTS
for i in date:
    start_time = time.time()
    # data
    data = data_full.loc[data_full.index > i, :]

    # reset index
    data.reset_index(inplace = True)
    data.drop('index', axis = 1, inplace = True)

    # fill nan values in the whole data set
    data.fillna(data.mean(), inplace = True)

    from sklearn.model_selection import train_test_split

    # divide data into train and test 
    data_train, data_test = train_test_split(
             data, test_size = 0.15, shuffle=False)  
    
    from sklearn.preprocessing import MinMaxScaler

    # data scaling  (including offer (y))
    sc_X = MinMaxScaler()
    data_train = sc_X.fit_transform(data_train)
    data_test = sc_X.transform(data_test)
    
    # divide features and labels
    X_train = data_train[:, 0:14] 
    y_train = data_train[:, -1]
    X_test = data_test[:, 0:14] 
    y_test = data_test[:, -1] 

    # divide data into train and test 
    X_train, X_val, y_train, y_val = train_test_split(
             X_train, y_train, test_size = 0.15, shuffle=False)

    # put data into correct shape
    X_train, y_train = split_data(X_train, y_train, steps)
    X_test, y_test = split_data(X_test, y_test, steps)
    X_val, y_val = split_data(X_val, y_val, steps)

    X_train = cut_data(X_train, batch_size)
    y_train = cut_data(y_train, batch_size)
    X_test = cut_data(X_test, batch_size)
    y_test = cut_data(y_test, batch_size)
    X_val = cut_data(X_val, batch_size)
    y_val = cut_data(y_val, batch_size)

    model = regressor_tunning()
    
    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = epochs,
                        shuffle = False, 
                        validation_data = (X_val, y_val))
    
    model.reset_states()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15 (inverso should not be used as scalling was made with the whole data set)
    y_pred = (y_pred * sc_X.data_range_[14]) + (sc_X.data_min_[14])
    y_test = (y_test * sc_X.data_range_[14]) + (sc_X.data_min_[14])

    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)
    
    y_pred_list.append(y_pred)
    
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)
    
    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    # Need to process data with spike occurences the same way as features
    data = pd.read_csv('Spike_binary_1std.csv', index_col = 0)

    # set predictive window according with tuning best results
    data = data.loc[data.index > i, :]

    # make sure shaded area will correspond to values outputed by LSTM
    data.reset_index(drop = True, inplace = True)

    # fill_nan is already made - so lets split data into test and train
    from sklearn.model_selection import train_test_split

    # divide data into train and test 
    shade_train, shade_test = train_test_split(
             data, test_size = 0.15, shuffle = False)

    # reset index of testing data
    shade_test.reset_index(drop = True, inplace = True)

    # function to split data into correct shape for RNN
    def split_data_shade(shade_test, steps):
        y_spike_occ = list()
        upper_lim = list()
        lower_lim = list()
        for i in range(steps, len(shade_test.index)):
            y_spike_occ.append(shade_test['spike_occurance'][i])
            upper_lim.append(shade_test['spike_upperlim'][i])
            lower_lim.append(shade_test['spike_lowerlim'][i])
        return np.array(y_spike_occ), np.array(upper_lim), np.array(lower_lim)

    # function to cut data set so it can be divisible by the batch_size
    def cut_data_shade(data, batch_size):
         # see if it is divisivel
        condition = data.shape[0] % batch_size
        if condition == 0:
            return data
        else:
            return data[: -condition]
    
    # shape y_spike_occ for the right size to compare results in normal and spike regions
    y_spike_occ, spike_upperlim, spike_lowerlim = split_data_shade(shade_test, steps)
    y_spike_occ = cut_data_shade(y_spike_occ, batch_size)

    # continue
    
    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)
    
    elapsed_time = time.time() - start_time

    time_count.append(elapsed_time)

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Epoch 1/100
37/37 [==============================] - 53s 1s/step - loss: 0.5342 - mse: 0.5342 - mae: 0.4698 - val_loss: 0.2124 - val_mse: 0.2124 - val_mae: 0.4599
Epoch 2/100
37/37 [==============================] - 47s 1s/step - loss: 0.1527 - mse: 0.1527 - mae: 0.3117 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0414
Epoch 3/100
37/37 [==============================] - 51s 1s/step - loss: 0.0868 - mse: 0.0868 - mae: 0.2363 - val_loss: 0.0263 - val_mse: 0.0263 - val_mae: 0.1604
Epoch 4/100
37/37 [==============================] - 62s 2s/step - loss: 0.0476 - mse: 0.0476 - mae: 0.1754 - val_loss: 0.0161 - val_mse: 0.0161 - val_mae: 0.1249
Epoch 5/100
37/37 [==============================] - 48s 1s/step - loss: 0.0249 - mse: 0.0249 - mae: 0.1261 - val_loss: 0.0036 - val_mse: 0.0036 - val_mae: 0.0567
Epoch 6/100
37/37 [==============================] - 55s 1s/step - loss: 0.0153 - mse: 0.0153 - mae: 0.0984 - val_loss: 0.0060 - val_mse: 0.0060 - val_mae: 0.0742
Epoch 7/100
37/37 [===

Epoch 49/100
37/37 [==============================] - 43s 1s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0199 - val_loss: 5.4916e-04 - val_mse: 5.4916e-04 - val_mae: 0.0169
Epoch 50/100
37/37 [==============================] - 42s 1s/step - loss: 0.0012 - mse: 0.0012 - mae: 0.0193 - val_loss: 5.4426e-04 - val_mse: 5.4426e-04 - val_mae: 0.0167
Epoch 51/100
37/37 [==============================] - 44s 1s/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0189 - val_loss: 5.5982e-04 - val_mse: 5.5982e-04 - val_mae: 0.0171
Epoch 52/100
37/37 [==============================] - 45s 1s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0200 - val_loss: 5.4271e-04 - val_mse: 5.4271e-04 - val_mae: 0.0166
Epoch 53/100
37/37 [==============================] - 43s 1s/step - loss: 9.9509e-04 - mse: 9.9509e-04 - mae: 0.0185 - val_loss: 5.4917e-04 - val_mse: 5.4917e-04 - val_mae: 0.0172
Epoch 54/100
37/37 [==============================] - 43s 1s/step - loss: 9.4052e-04 - mse: 9.4052e-04 - mae: 0.0180 - val_loss: 5.6

Epoch 96/100
37/37 [==============================] - 42s 1s/step - loss: 8.8898e-04 - mse: 8.8898e-04 - mae: 0.0176 - val_loss: 4.7853e-04 - val_mse: 4.7853e-04 - val_mae: 0.0153
Epoch 97/100
37/37 [==============================] - 44s 1s/step - loss: 8.6936e-04 - mse: 8.6936e-04 - mae: 0.0171 - val_loss: 4.8384e-04 - val_mse: 4.8384e-04 - val_mae: 0.0159
Epoch 98/100
37/37 [==============================] - 42s 1s/step - loss: 8.5443e-04 - mse: 8.5443e-04 - mae: 0.0175 - val_loss: 4.7029e-04 - val_mse: 4.7029e-04 - val_mae: 0.0155
Epoch 99/100
37/37 [==============================] - 41s 1s/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0182 - val_loss: 4.6555e-04 - val_mse: 4.6555e-04 - val_mae: 0.0148
Epoch 100/100
37/37 [==============================] - 42s 1s/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0181 - val_loss: 4.6011e-04 - val_mse: 4.6011e-04 - val_mae: 0.0147


C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


...

Epoch 1/100
25/25 [==============================] - 31s 1s/step - loss: 0.7373 - mse: 0.7373 - mae: 0.5070 - val_loss: 0.1257 - val_mse: 0.1257 - val_mae: 0.3513
Epoch 2/100
25/25 [==============================] - 30s 1s/step - loss: 0.2120 - mse: 0.2120 - mae: 0.3669 - val_loss: 0.0025 - val_mse: 0.0025 - val_mae: 0.0260
Epoch 3/100
25/25 [==============================] - 30s 1s/step - loss: 0.1560 - mse: 0.1560 - mae: 0.3158 - val_loss: 0.0354 - val_mse: 0.0354 - val_mae: 0.1828
Epoch 4/100
25/25 [==============================] - 32s 1s/step - loss: 0.0956 - mse: 0.0956 - mae: 0.2446 - val_loss: 0.0035 - val_mse: 0.0035 - val_mae: 0.0395
Epoch 5/100
25/25 [==============================] - 31s 1s/step - loss: 0.0792 - mse: 0.0792 - mae: 0.2273 - val_loss: 0.0277 - val_mse: 0.0277 - val_mae: 0.1631
Epoch 6/100
25/25 [==============================] - 30s 1s/step - loss: 0.0478 - mse: 0.0478 - mae: 0.1750 - val_loss: 0.0388 - val_mse: 0.0388 - val_mae: 0.1944
Epoch 7/100
25/25 [===

25/25 [==============================] - 32s 1s/step - loss: 5.8729e-04 - mse: 5.8729e-04 - mae: 0.0150 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0166
Epoch 50/100
25/25 [==============================] - 30s 1s/step - loss: 5.7679e-04 - mse: 5.7679e-04 - mae: 0.0149 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0167
Epoch 51/100
25/25 [==============================] - 30s 1s/step - loss: 5.7552e-04 - mse: 5.7552e-04 - mae: 0.0149 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0167
Epoch 52/100
25/25 [==============================] - 30s 1s/step - loss: 5.7926e-04 - mse: 5.7926e-04 - mae: 0.0149 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0167
Epoch 53/100
25/25 [==============================] - 31s 1s/step - loss: 5.7828e-04 - mse: 5.7828e-04 - mae: 0.0149 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0165
Epoch 54/100
25/25 [==============================] - 30s 1s/step - loss: 5.6976e-04 - mse: 5.6976e-04 - mae: 0.0148 - val_loss: 0.0016 - val_mse: 0.0016

Epoch 97/100
25/25 [==============================] - 31s 1s/step - loss: 5.2991e-04 - mse: 5.2991e-04 - mae: 0.0140 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0161
Epoch 98/100
25/25 [==============================] - 31s 1s/step - loss: 5.3402e-04 - mse: 5.3402e-04 - mae: 0.0140 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0161
Epoch 99/100
25/25 [==============================] - 30s 1s/step - loss: 5.2935e-04 - mse: 5.2935e-04 - mae: 0.0140 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0161
Epoch 100/100
25/25 [==============================] - 30s 1s/step - loss: 5.2847e-04 - mse: 5.2847e-04 - mae: 0.0139 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0162


C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Epoch 1/100
18/18 [==============================] - 23s 1s/step - loss: 0.5697 - mse: 0.5697 - mae: 0.4875 - val_loss: 0.0222 - val_mse: 0.0222 - val_mae: 0.1441
Epoch 2/100
18/18 [==============================] - 22s 1s/step - loss: 0.2039 - mse: 0.2039 - mae: 0.3607 - val_loss: 0.0482 - val_mse: 0.0482 - val_mae: 0.2174
Epoch 3/100
18/18 [==============================] - 21s 1s/step - loss: 0.1928 - mse: 0.1928 - mae: 0.3510 - val_loss: 0.1981 - val_mse: 0.1981 - val_mae: 0.4440
Epoch 4/100
18/18 [==============================] - 21s 1s/step - loss: 0.1477 - mse: 0.1477 - mae: 0.3066 - val_loss: 0.0380 - val_mse: 0.0380 - val_mae: 0.1931
Epoch 5/100
18/18 [==============================] - 22s 1s/step - loss: 0.1057 - mse: 0.1057 - mae: 0.2599 - val_loss: 0.0130 - val_mse: 0.0130 - val_mae: 0.1116
Epoch 6/100
18/18 [==============================] - 23s 1s/step - loss: 0.0786 - mse: 0.0786 - mae: 0.2245 - val_loss: 0.0510 - val_mse: 0.0510 - val_mae: 0.2241
Epoch 7/100
18/18 [===

In [ ]:
results = pd.DataFrame({                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
    
                        'time': time_count})

y_pred = pd.DataFrame({'dates': date,
                       'Predicitons': y_pred_list})

y_pred.to_csv('Pedictions_LSTM_5_prediction_window.csv')

In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

In [ ]:
results.min()

In [ ]:
!pip install matplotlib

import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

dates_labels = ['12 ',
                '10 ',
                '8 ',
                '6 ',
                '4 ',
                '2 ']

plt.figure(figsize=(10,4))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('LSTM: Averaged RMSE for different\n predictive windows')
plt.plot(rmse_gen, label = 'Overall error')
plt.plot(rmse_spi, label = 'Spike regions')
plt.plot(rmse_nor, label = 'Normal regions')
plt.legend()
plt.ylabel('RMSE (£/MWh)')
plt.xlabel('Predictive window (in months)')
plt.xticks([0,1,2,3,4,5], dates_labels)
plt.tight_layout()
plt.savefig('RMSE_predictive_window.png')

plt.figure(figsize=(10,4))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('LSTM: Averaged MAE for different\n predictive windows')
plt.plot(mae_gen, label = 'Overall error')
plt.plot(mae_spi, label = 'Spike regions')
plt.plot(mae_nor, label = 'Normal regions')
plt.legend()
plt.ylabel('MAE (£/MWh)')
plt.xlabel('Predictive window (in months)')
plt.xticks([0,1,2,3,4,5], dates_labels)
plt.tight_layout()
plt.savefig('MAE_predictive_window.png')